In [1]:
from importnb import Notebook, reload, Lazy, Partial, load_ipython_extension, unload_ipython_extension
from pathlib import Path
import shutil, os, functools, sys
from pytest import fixture, mark
from importnb.loader import __IPYTHON__
import warnings
print(__IPYTHON__)

True


In [2]:
try:
    from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell, writes
except:
    from functools import partial
    from json import dumps 
    writes = partial(dumps, indent=2)
    new_notebook=dict
    def new_cell(type, source): return {
        'cell_type': type, 'source': list(map("{}\n".format, source.splitlines())), 'metadata': {}}
    new_markdown_cell = partial(new_cell, 'markdown')
    new_code_cell = partial(new_cell, 'code')

In [3]:
def atestnotebook(str='foo') -> str:
    """Stringify a new notebook to test with a simple set of instructions that may be formatter.
    
    >>> assert isinstance(atestnotebook(), str)
    """
    return writes(new_notebook(cells=[
            new_markdown_cell("""This is the docstring.
                
                >>> assert True
            """),
            new_code_cell("""foo = 42\nassert {}\nbar= 100""".format(str)),
            new_code_cell("""print(foo)"""),
            new_markdown_cell("""Markdown paragraph"""),
    ]))

# Test Single File Modules

Single file modules mimic common Untitled notebooks.  An author should be able to trivially import notebooks in their working directory.

## Single File Fixtures

In [4]:
@fixture(scope='function')
def single_file(request):
    """A fixture to write a new notebook to disk and delete after each function call."""
    name = Path('foobar.ipynb')
    with name.open('w') as file:
        file.write(atestnotebook())
    request.addfinalizer(functools.partial(os.remove, str(name)))
    return file

Each time a file is imported we should clear up the sys path to reset our imports and assure the validity of our tests.

In [5]:
@fixture
def nb(single_file, request):
    def clean_sys():
        import sys
        del sys.modules['foobar']
        sys.path_importer_cache.clear()
    request.addfinalizer(clean_sys)

`importnb`'s most generic use is as a context manager.  `with Notebook()` will update the `sys.path_hooks` to import notebooks as modules.

In [6]:
def test_single_file_with_context(nb):
    with Notebook():
        import foobar
    assert foobar.foo == 42 and foobar.bar == 100
    
    validate_reload(foobar)

Add tests for failure condition

In [7]:
three_point_seven = mark.skipif(sys.version_info.major == 3 and sys.version_info.minor >= 3.7, reason="""There is a new docstring parameter in the Module ast.""")


In [8]:
@three_point_seven
def test_from_filename(single_file):
    foobar = Notebook(stdout=True).from_filename('foobar.ipynb')
    assert foobar.foo == 42 and foobar.bar == 100
    assert foobar.__name__ != '__main__'
    assert foobar.__output__.stdout
    assert foobar.__doc__.strip().startswith("""This is the docstring.""")

In [9]:
@three_point_seven
def test_with_doctest(single_file):
    foobar = Notebook(stdout=True)('foobar.ipynb')
    import doctest
    test = doctest.testmod(foobar)
    assert test.attempted
    assert not test.failed

In [26]:
def test_from_filename_main(single_file):
    foobar = Notebook('__main__', stdout=True).from_filename('foobar.ipynb')
    assert foobar.foo == 42 and foobar.bar == 100
    assert foobar.__name__ == '__main__'
    assert foobar.__output__.stdout
    

In [11]:
def test_parameterize(nb):
    with Notebook():
        import foobar
    
    from importnb.parameterize import Parameterize
    
    f = Parameterize(foobar)
    
    foobar = f()
    
    assert foobar.foo == 42 and foobar.bar == 100
    
    foobar = f(foo="something", bar=0)
    assert foobar.foo == "something" and foobar.bar == 0

In [12]:
def test_python_file(nb):
    with Notebook(stdout=True) as out:
        from pyimport import foobar
    assert foobar.foo == 42 and foobar.bar == 100
    assert out.stdout == "42\n"
    validate_reload(foobar)

In [13]:
def test_single_file_with_capture(nb):
    # I don't think i can test stderr with pytest
    with Notebook(stdout=True, stderr=True) as out:
        import foobar
    assert foobar.foo == 42 and foobar.bar == 100
    assert out.stdout
#     assert out.stderr
    assert out.stdout == "42\n"
    
    validate_reload(foobar)

In [14]:
def test_capturer():
    from importnb.capture import capture_output
    if __IPYTHON__:
        from IPython.utils.capture import capture_output as ipython_version
        assert capture_output is ipython_version

In [15]:
@mark.skipif(sys.version_info.minor==4, reason="""Requires > python 3.5""")
def test_single_file_with_lazy(nb):
    from importnb.capture import capture_output    
    with Lazy() as out:
        import foobar
    assert not out.stdout
    with capture_output() as out:
        foobar.foo
    assert out.stdout
    validate_reload(foobar)

Each time we test a notebook import we should test the ability to reload the module.  `importnb` expresses the ability to use the normal Python import system, and a notebook must reload for interactive development.

In [16]:
def validate_reload(module):
    try:
        reload(module)
        assert False, """The reload should have fail."""
    except:
        assert True, """Cannot reload a file outside of a context manager"""

    with Notebook():
        assert reload(module)

A notebook will not import without the context manager or [IPython extension](#IPython-extension).

In [17]:
@mark.xfail
def test_single_file_without_context():
    import foobar

In the `__main__` context, relative imports are not allowed. 

In [18]:
@mark.xfail
def test_single_file_relative(single_file):
    with Notebook():
        from . import foobar

Commonly, we use the `try` statement to allow the ability to use relative imports in a package while developing interactively.

    try:
        from . import a_module
    except:
        import a_module

## IPython extension

In general, an author would use IPython sugar to load an extension

    %load_ext importnb
    
For testing purposes we use the explicit functions to create the extensions

In [19]:
@fixture
def extension(nb, request):
    load_ipython_extension()
    request.addfinalizer(unload_ipython_extension)

In [20]:
def test_single_with_extension(extension):
    import foobar
    assert foobar.foo == 42 and foobar.bar == 100

In [21]:
@fixture
def single_directory(request):
    root = Path('a_test_package')
    try:
        root.mkdir(exist_ok=True)
    except TypeError:
        #py34
        try:
            root.mkdir(parents=True)
        except FileExistsError: ...
    with (root / 'foobar.ipynb').open('w') as file:
        file.write(atestnotebook())
    with (root / 'failure.ipynb').open('w') as file:
        file.write(atestnotebook('False'))
    with (root / 'py.py').open('w') as file:
        file.write("""from . import foobar\nbaz = 'foobar'""")
    request.addfinalizer(functools.partial(shutil.rmtree, str(root)))
    return root

In [22]:
def test_package(single_directory):
    with Notebook():
        from a_test_package import foobar, py
        
    assert foobar.foo == 42 and foobar.bar == 100
    assert py.baz == 'foobar'
    assert py.foobar is foobar
    validate_reload(foobar)

In [23]:
@mark.xfail
def test_package_failure(single_directory):
    with Notebook():
        from a_test_package import failure

## Partial Imports.

In [24]:
def test_package_failure_partial(single_directory):
    with Partial():
        from a_test_package import failure
        
    assert isinstance(failure.__exception__, AssertionError), """
    The wrong error was returned likely because of importnb."""

    from traceback import print_tb
    from io import StringIO
    s = StringIO()
    with open(failure.__file__, 'r') as f:
        line = list(i for i, line in enumerate(f.read().splitlines()) if 'assert False' in line)[0] + 1
    print_tb(failure.__exception__.__traceback__, file=s)
    assert """a_test_package/failure.ipynb", line {}, in <module>\n""".format(line) in s.getvalue(), """Traceback is not satisfied"""